In [3]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3 # connect S3 bucket
import pandas as pd

sm_boto3 = boto3.client('sagemaker') # client that can commnicate with S3 bucket
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'aistarterssagemakerbucket'
print('Using bucket ' + bucket)


c:\Users\kevin\OneDrive\Desktop\Lovely's 100 Days Coding Journey March 6 2025 - June 14, 2025\E2E ML Project AWS Sagemaker\aistarters1stMLProject\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/06/25 14:48:36] INFO     Found credentials in shared credentials file: ~/.aws/credentials   credentials.py:1352

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\kevin\AppData\Local\sagemaker\sagemaker\config.yaml


[03/06/25 14:48:38] INFO     Found credentials in shared credentials file: ~/.aws/credentials   credentials.py:1352

Using bucket aistarterssagemakerbucket


In [20]:
df = pd.read_csv('train_MobilePriceClassification.csv')
df.head()


,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [21]:
print(df.columns)

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')


# Feature Engineering 

In [10]:
# ['Low_Risk','High_Risk], [0,1]
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [22]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [23]:
df.shape

(2000, 21)

In [24]:
# find percentage of values missing in each column
df.isnull().mean()*100

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [25]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [26]:
# remove last features - price_range
label = features.pop(-1)
label

'price_range'

In [27]:
# divide into independent and dependent features
x=df[features]
y=df[label]

In [33]:
# independent features
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [34]:
x.shape

(2000, 20)

In [35]:
# dependent features
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [36]:
y.shape

(2000,)

In [38]:
# number of categories in the dependent feature amd equally proportioned --> balanced dataset
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [39]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=0)

In [40]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1700, 20) (300, 20) (1700,) (300,)


In [41]:
# add a new column (label) to a Pandas DataFrame (trainX) and assign it the values from y_train.
trainX = pd.DataFrame(X_train)
trainX[label] = y_train

testX = pd.DataFrame(X_test)    
testX[label] = y_test

In [42]:
print(trainX.shape, testX.shape)

(1700, 21) (300, 21)


In [43]:
trainX.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
1452,1450,0,2.1,0,1,0,31,0.6,114,5,...,1573,1639,794,11,5,9,0,1,1,1
1044,1218,1,2.8,1,3,0,39,0.8,150,7,...,1122,1746,1667,10,0,12,0,0,0,1
1279,1602,0,0.6,0,12,0,58,0.4,170,1,...,1259,1746,3622,17,2,17,0,1,1,3
674,1034,0,2.6,1,2,1,45,0.3,190,3,...,182,1293,969,15,1,7,1,0,0,0
1200,530,0,2.4,0,1,0,32,0.3,88,6,...,48,1012,959,17,7,6,0,1,0,0


In [44]:
trainX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [45]:
#  count the number of missing (NaN) values in each column of the testX DataFrame.
testX.isnull().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [46]:
trainX.to_csv('train-V-1.csv', index=False)
testX.to_csv('test-V-1.csv', index=False)

In [47]:
bucket

'aistarterssagemakerbucket'

In [49]:
# send data to S3 bucket. SageMaker will take training data from S3.
sk_prefix = 'sagemaker/mobile_price_classification/sklearncontainer'
trainpath = sess.upload_data(
    path='train-V-1.csv', bucket=bucket,
    key_prefix=sk_prefix)

testpath = sess.upload_data(
    path='test-V-1.csv', bucket=bucket,
    key_prefix=sk_prefix)

print(trainpath)
print(testpath)

s3://aistarterssagemakerbucket/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
s3://aistarterssagemakerbucket/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


Now we have finished Data Ingestion phase.

In [51]:
# create a script file that will be executed by the SageMaker training job.
%%writefile script.py 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd

# Function to load the model   
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
    
if __name__ == "__main__":
    # whenever training model in sagemaker, some default arguments are required by AWS sagemaker

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # Argparser are some arguments of RandomForestClassifier
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train-V-1.csv")
    parser.add_argument("--test-file", type=str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (85%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (15%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 3,n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Writing script.py


In [54]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

# create an instance in sagemaker by assigning a machien
sklearn_estimator = SKLearn(
    entry_point="script.py",
    # role="arn:aws:iam::396608809446:user/aistarters",
    role="arn:aws:iam::396608809446:role/service-role/AmazonSageMaker-ExecutionRole-20250309T174684",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [55]:
# launch training job, with asynchronous call. Unsynchroneous call means that the code will not wait for the training job to complete.
# sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=False)
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)
# sklearn_estimator.fit({"train": datapath}, wait=True)

[03/09/25 17:49:31] INFO     SageMaker Python SDK will collect telemetry to help us better  telemetry_logging.py:91
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[03/09/25 17:49:35] INFO     Creating training-job with name:                                       session.py:1042
                             RF-custom-sklearn-2025-03-09-16-49-31-044                                             

2025-03-09 16:49:38 Starting - Starting the training job...
2025-03-09 16:50:13 Downloading - Downloading input data...
2025-03-09 16:50:44 Downloading - Downloading the training image.....2025-03-09 16:51:28,668 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-03-09 16:51:28,673 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-09 16:51:28,722 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-03-09 16:51:29,012 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-09 16:51:29,025 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-09 16:51:29,039 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-03-09 16:51:29,048 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
    

In [56]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2025-03-09 16:51:52 Starting - Preparing the instances for training
2025-03-09 16:51:52 Downloading - Downloading the training image
2025-03-09 16:51:52 Training - Training image download completed. Training in progress.
2025-03-09 16:51:52 Uploading - Uploading generated training model
2025-03-09 16:51:52 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-1-396608809446/RF-custom-sklearn-2025-03-09-16-49-31-044/output/model.tar.gz


In [57]:
# model output
artifact

's3://sagemaker-us-east-1-396608809446/RF-custom-sklearn-2025-03-09-16-49-31-044/output/model.tar.gz'

In [58]:
# Deploy the model to an endpoint
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

# moving to a folder location starting with Custom-sklearn-model-
model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name =  model_name,
    model_data=artifact,
    role="arn:aws:iam::396608809446:role/service-role/AmazonSageMaker-ExecutionRole-20250309T174684",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [59]:
model

In [61]:
model_name

'Custom-sklearn-model-2025-03-09-16-59-45'

In [60]:
# Endpoint Deployment
endpoint_name = "custom-sklearn-endpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Deploying model to endpoint: ", endpoint_name)
#print("EndpointName={}".format(endpoint_name))
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

Deploying model to endpoint:  custom-sklearn-endpoint-2025-03-09-17-01-59


[03/09/25 18:02:04] INFO     Creating model with name: Custom-sklearn-model-2025-03-09-16-59-45     session.py:4094

[03/09/25 18:02:06] INFO     Creating endpoint-config with name                                     session.py:5937
                             custom-sklearn-endpoint-2025-03-09-17-01-59                                           

                    INFO     Creating endpoint with name                                            session.py:4759
                             custom-sklearn-endpoint-2025-03-09-17-01-59                                           

------!

In [69]:
# return type is SKLearnPredictor which is a wrapper around the endpoint
type(predictor)


sagemaker.sklearn.model.SKLearnPredictor

In [70]:
predictor 

In [62]:
endpoint_name

'custom-sklearn-endpoint-2025-03-09-17-01-59'

In [64]:
testX

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
405,1454,1,0.5,1,1,0,34,0.7,83,4,...,250,1033,3419,7,5,5,1,1,0,3
1190,1092,1,0.5,1,10,0,11,0.5,167,3,...,468,571,737,14,4,11,0,1,0,0
1132,1524,1,1.8,1,0,0,10,0.6,174,4,...,154,550,2678,16,5,13,1,0,1,2
731,1807,1,2.1,0,2,0,49,0.8,125,1,...,337,1384,1906,17,13,13,0,1,1,2
1754,1086,1,1.7,1,0,1,43,0.2,111,6,...,56,1150,3285,11,5,17,1,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,1197,1,0.5,1,9,0,20,0.1,129,5,...,214,1710,2885,19,7,14,1,1,1,2
982,1035,0,0.6,1,2,1,44,0.5,99,3,...,236,1960,3506,14,5,12,1,1,0,3
723,1271,0,0.5,1,2,0,54,0.1,159,8,...,613,650,990,14,11,6,0,1,1,0
1487,761,1,0.5,0,9,0,3,0.8,196,7,...,1325,1945,2248,19,3,15,1,1,1,2


In [63]:
testX[features][0:2].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0]]

In [65]:
print(predictor.predict(testX[features][0:2].values.tolist()))

[3 0]


In [71]:
# delete endpoint to avoid incurring charges
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'f8595daf-5d22-4ffc-89ef-680e7fc7794a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f8595daf-5d22-4ffc-89ef-680e7fc7794a',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 09 Mar 2025 17:13:49 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}